In [2]:
import keras
import numpy as np
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


In [3]:
# Normalizing text
normalized_text = text.replace('\n', ' ').replace('preface   ', '').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ')
normalized_text[0:300]

'supposing that truth is a woman--what then? is there not ground for suspecting that all philosophers, in so far as they have been dogmatists, have failed to understand women--that the terrible seriousness and clumsy importunity with which they have usually paid their addresses to truth, have been un'

In [5]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(normalized_text) - maxlen, step):
    sentences.append(normalized_text[i: i + maxlen])
    next_chars.append(normalized_text[i + maxlen])
print('Number of sequences:', len(sentences))

chars = sorted(list(set(normalized_text)))
print('Unique characters:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)
print('Vectorization...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 199598
Unique characters: 56
Vectorization...


In [11]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [64]:
# Add some temperature to the predictions for more uncommon predictions
def sample(preds, temperature=1.0):
    # Add some temperature to the prediction array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    # "Rolls a dice" with the probabilities in the array and sets the chosen index to 1 and others to 0
    probas = np.random.multinomial(1, preds, 1)
    # Returns the chosen index 
    return np.argmax(probas)


In [65]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(normalized_text) - maxlen - 1)
    generated_text = normalized_text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

epoch 1
1560/1560 [==============================] - 306s 196ms/step - loss: 1.9236
--- Generating with seed: "urther-sighted sympathy:--we see how man dwarfs himself, how"
------ temperature: 0.2
urther-sighted sympathy:--we see how man dwarfs himself, how the supersible of the such a spirit to the such a strough the supersible of the such a mean of the such a such a such a presens of the such conself of the supersible the the mast the superstine of the supersible of the such the regard the supersible the such a the resull of the most the such a such all the supersible of the same as the such a man the supersible of the such all the best be the sur------ temperature: 0.5
uch a man the supersible of the such all the best be the surjection of the free of he were the prespens, a dearist as the regard of the strought of the great german of a freat the strolally the regard the pressivist. man for the conficised the be the supersible that the great be courted the supersing so all the all th

KeyboardInterrupt: 